<a href="https://colab.research.google.com/github/SachinthaWeerasinghe/Deep-Learning-Mini-Project-03-PS-2019-259/blob/main/PS_2019_259_English_Sinhala_Tanslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **English to Sinhala Translattion System using Transformer Neural Network**
# **COSC 44323 - Introduction To Deep Learning**

**Mini Project No. 03**

**Student Number: PS/2019/259**

**Name: S.A. WEERASINGHE**

**IMPORT NECESSARY LIBRARIES**

In [ ]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

**MOUNT GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**READ THE DATASET**

In [ ]:
text_file = "/content/drive/MyDrive/PS-2019-259-COSC44323-MiniProject03/EnglishSinhalaDataset-PS-2019-259.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

Go.	යන්න.
Hi.	ආයුබෝවන්.
Run.	දුවන්න.
Who?	කව්ද?
Wow!	වාව්!
Fire.	ගින්නක්.
Help.	උදව්.
Hide.	සඟවන්න.
Jump.	පනින්න.
Stay.	රැඳී සිටින්න.
Stop.	නවත්වන්න.
Wait.	ඉන්න.
Begin.	ආරම්භය.
Go on.	දිගටම යන්න.
Hello!	හෙලෝ!
Hurry!	ඉක්මන් කරන්න!
I hid.	මම සැඟවී සිටියෙමි.
I ran.	මම දිව්වා.
I try.	මම උත්සාහ කරමි.
I won.	මම දිනුවා.


In [ ]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.	චොක්ලට් යනු වැනිලා බව මිනිසුන්ට ඒත්තු ගැන්වීමට ඔබ කොතරම් උත්සාහ කළත්, එය වැනිලා බව ඔබට සහ තවත් කිහිප දෙනෙකුට ඒත්තු ගැන්විය හැකි වුවද, එය තවමත් චොකලට් වනු ඇත.
In 1969, Roger Miller recorded a song called "You Don't Want My Love." Today, this song is better known as "In the Summer Time." It's the first song he wrote and sang that became popular.	1969 දී රොජර් මිලර් "ඔබට මගේ ආදරය අවශ්‍ය නැත" නමින් ගීතයක් පටිගත කළේය. අද මෙම ගීතය වඩාත් ප්‍රචලිත වන්නේ "ගිම්හානයේ" යනුවෙනි. එය ඔහු ලියූ සහ ගායනා කළ පළමු ගීතය ජනප්‍රිය විය.
A child who is a native speaker usually knows many things about his or her language that a non-native speaker who has been studying for years still does not know and perhaps will never know.	ස්වදේශික කථිකයෙකු වන දරුවෙකු සාමාන්‍යයෙන් ඔහුගේ හෝ ඇයගේ භාෂාව පිළිබඳ බොහෝ දේ දන්නා අතර එය වසර ගණනාවක් තිස්ස

**SPLIT THE ENGLISH AND SINHALA TRANSLATION PAIRS**

In [ ]:
text_pairs = []
for line in lines:
    english, sinhala = line.split("\t")
    sinhala = "[start] " + sinhala + " [end]"
    text_pairs.append((english, sinhala))
for i in range(3):
  print(random.choice(text_pairs))

('Tom changed his name.', '[start] ටොමස් ඔහුගේ නම වෙනස් කළේය. [end]')
("If I were you, I'd buy that one.", '[start] මම ඔබ නම්, මම එය මිලදී ගන්නෙමි. [end]')
('I need you to see this.', '[start] මට ඔයා මේක බලන්න ඕන. [end]')


**RANDOMIZE THE DATA**


In [ ]:
import random
random.shuffle(text_pairs)

**SPITING THE DATASET INTO TRAINING, TESTING AND VALIDATION DATA**

In [ ]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))
print("Total size of the dataset:",len(train_pairs)+len(val_pairs)+len(test_pairs))

Total sentences: 138902
Training set size: 97232
Validation set size: 20835
Testing set size: 20835
Total size of the dataset: 138902


**REMOVING PUNCTUATIONS**

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [ ]:
f"{3+5}"

**VECTORIZATION THE ENGLISH AND SINHALA TEXT PAIRS**

In [ ]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]

In [ ]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

**PREPARATION OF THE DATASET FRO THE TRANSLATION**

In [ ]:
batch_size = 64
def format_dataset(eng, sin):
    eng = source_vectorization(eng)
    sin = target_vectorization(sin)
    return ({
        "english": eng,
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])
def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")
inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[  2, 285, 132, ...,   0,   0,   0],
       [ 39,   2, 282, ...,   0,   0,   0],
       [335,  30,   7, ...,   0,   0,   0],
       ...,
       [ 52,  62, 202, ...,   0,   0,   0],
       [  3, 333,  10, ...,   0,   0,   0],
       [  5, 104,  99, ...,   0,   0,   0]]), 'sinhala': array([[   2,    4,  333, ...,    0,    0,    0],
       [   2,    5,   21, ...,    0,    0,    0],
       [   2,   20,  295, ...,    0,    0,    0],
       ...,
       [   2,  283,   27, ...,    0,    0,    0],
       [   2,  862, 3968, ...,    0,    0,    0],
       [   2,    7,  197, ...,    0,    0,    0]])}, array([[   4,  333,  399, ...,    0,    0,    0],
       [   5,   21,  694, ...,    0,    0,    0],
       [  20,  295, 7949, ...,    0,    0,    0],
       ...,
       [ 283,   27,  506, ...,    0,    0,    0],
       [ 862, 3968, 1433, ...,    0,    0,    0],
       [   7,  197, 8931, ...

**TRANSFORMER ENCODER IMPLEMENTED AS A SUBCLASSED LAYER**

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**THE TRANSFORMER DECODER**

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


**POSITIONAL ENCODEING**

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**END-TO-END TRANSFORMER**

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

**TRAINING THE TRANSFORMER NEURAL NETWORK**

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=40, validation_data=val_ds)

Epoch 1/40
1520/1520 [==============================] - 123s 75ms/step - loss: 4.2285 - accuracy: 0.4214 - val_loss: 3.3254 - val_accuracy: 0.5124
Epoch 2/40
1520/1520 [==============================] - 106s 70ms/step - loss: 3.2785 - accuracy: 0.5176 - val_loss: 2.9444 - val_accuracy: 0.5563
Epoch 3/40
1520/1520 [==============================] - 116s 76ms/step - loss: 2.9957 - accuracy: 0.5545 - val_loss: 2.8368 - val_accuracy: 0.5759
Epoch 4/40
1520/1520 [==============================] - 104s 69ms/step - loss: 2.8519 - accuracy: 0.5762 - val_loss: 2.7831 - val_accuracy: 0.5848
Epoch 5/40
1520/1520 [==============================] - 104s 69ms/step - loss: 2.7627 - accuracy: 0.5923 - val_loss: 2.7631 - val_accuracy: 0.5935
Epoch 6/40
1520/1520 [==============================] - 104s 68ms/step - loss: 2.6996 - accuracy: 0.6037 - val_loss: 2.7403 - val_accuracy: 0.5978
Epoch 7/40
1520/1520 [==============================] - 104s 69ms/step - loss: 2.6391 - accuracy: 0.6151 - val_loss: 2

**MANUAL TESTING OF THE TRANSLATION MODEL WITH 20 NEW SENTENCES**

In [ ]:
import numpy as np
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I'm always busy with my homework.
[start] මම නිතරම මගේ ගෙදර වැඩ සමඟ කාර්යබහුලයි [end]
-
Are you going to come visit me?
[start] ඔයා මාව බලන්න එයි [end]
-
I didn't make anything to eat.
[start] මම කිසිම දෙයක් කන්න ඕන වුණේ නැහැ [end]
-
Tom has made an about-face.
[start] ටොම්ට [UNK] සාදා ඇත [end]
-
When he was a child, his ambition was to be an English teacher.
[start] ඔහුගේ පුතා ගුරුවරියක් වූයේ ඔහු ඉංග්‍රීසි පමණක් [UNK] [end]
-
I received a letter written in English yesterday.
[start] ඊයේ මම ලිපියක් භාෂාවෙන් ලිපියක් ලියා තිබුණා [end]
-
I saw the car hit a man.
[start] මම දැක්කා කාර් එක කොල්ලෙක් [end]
-
I've finished translating everything you asked me to translate.
[start] ඔබ මගෙන් පරිවර්තනය කළ සෑම දෙයක්ම මම ඉල්ලා අවසන් කරන බව මම මකා දමා ඇත [end]
-
He abandoned his wife and children.
[start] ඔහු තම බිරිඳ සහ දරුවන් අතහැර ගියේය [end]
-
Study these sentences.
[start] මෙම [UNK] ඉගෙන ගන්න [end]
-
I'm tied up right now.
[start] මම දැන් මගේ ගෑස් කැඩුවා [end]
-
Tom sent me a message.
[start] 